# Agent Trajectory

Agents take actions in pursuit of a goal. "Trajectories" record the intermediate steps
taken by the agent. You can use the the `TrajectoryEvalChain` to grade how effective these steps
are at achieving the correct response.

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.evaluation import TrajectoryEvalChain

llm = ChatOpenAI(model="gpt-4", temperature=0)
chain = TrajectoryEvalChain.from_llm(llm)

## Capturing Trajectory

To return the trajectory, initialize an agent with `return_intermediate_steps=True`.

In [2]:
import os
from langchain.tools import tool
from langchain.agents import AgentType, initialize_agent
from pydantic import HttpUrl
import subprocess
from urllib.parse import urlparse

@tool
def ping(url: HttpUrl, return_error: bool) -> str:
    """Ping the fully specified url. Must include https:// in the url."""
    hostname = urlparse(str(url)).netloc
    completed_process = subprocess.run(['ping', '-c', '1', hostname], capture_output=True, text=True)
    output = completed_process.stdout
    if return_error and completed_process.returncode != 0:
        return completed_process.stderr
    return output

@tool
def trace_route(url: HttpUrl, return_error: bool) -> str:
    """Trace the route to the specified url. Must include https:// in the url."""
    hostname = urlparse(str(url)).netloc
    completed_process = subprocess.run(['traceroute', hostname], capture_output=True, text=True)
    output = completed_process.stdout
    if return_error and completed_process.returncode != 0:
        return completed_process.stderr
    return output



llm = ChatOpenAI(model="gpt-3.5-turbo-0613", temperature=0)
agent = initialize_agent(
    llm=llm,
    tools=[ping, trace_route],
    agent=AgentType.OPENAI_MULTI_FUNCTIONS,
    return_intermediate_steps=True # IMPORTANT!
)

result = agent("What's the latency like for https://langchain.com?")

## Evaluate Trajectory

Pass the input, trajectory, and output to the `evaluate_agent_trajectory` function.

In [3]:
evaluation_result = chain.evaluate_agent_trajectory(
    prediction=result["output"],
    input=result["input"],
    agent_trajectory=result["intermediate_steps"],
)
evaluation_result["grade"]

KeyError: 'grade'